In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# load in Breast Cancer Wisconsin (Diagnostic) Data Set
data_path = '/content/drive/My Drive/Colab Notebooks/LogisticRegressionforBreastCancerPredictions/data.csv'
df = pd.read_csv(data_path)

In [4]:
# display info about dataset
df.info()
df.head()
#df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [5]:
# decided to use tumor radius, texture, and smoothness as features
features_selected = ['radius_mean', 'texture_mean', 'smoothness_mean', 'diagnosis']
all = df[features_selected]
all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   radius_mean      569 non-null    float64
 1   texture_mean     569 non-null    float64
 2   smoothness_mean  569 non-null    float64
 3   diagnosis        569 non-null    object 
dtypes: float64(3), object(1)
memory usage: 17.9+ KB


In [51]:
# Using 80% of dataset for training and 20% for testing
all_train = all.sample(frac=0.8, random_state=1)
all_test = all.drop(all_train.index)

X = all[['radius_mean', 'texture_mean', 'smoothness_mean']]
Y = all['diagnosis']

X_train = all_train[['radius_mean', 'texture_mean', 'smoothness_mean']]
Y_train = all_train['diagnosis']

X_test = all_test[['radius_mean', 'texture_mean', 'smoothness_mean']]
Y_test = all_test['diagnosis']

Y = np.where(Y == 'M', 1, 0)
Y_train = np.where(Y_train == 'M', 1, 0)
Y_test = np.where(Y_test == 'M', 1, 0)

In [52]:
# Add interaction and polynomial terms into the training dataset
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=3)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

In [53]:
# Use z-score normalization to make sure each feature contributes equally to model
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

In [54]:
# check how many features are in model
rows, columns = X_train_scaled.shape
print(rows, columns)

weights = np.zeros(columns)
b = 0.0

455 20


In [62]:
# develop functions to implement training

def Sigmoid(x):
  return 1 / (1 + np.exp(-x))



def CostFunction(X, Y, weights, b):
  cost = 0
  m = len(Y)

  for i in range(m):
    x = X[i]
    y = Y[i]

    h = Sigmoid(np.dot(x, weights) + b)
    cost += -y * np.log(h) - (1 - y) * np.log(1 - h)

  cost /= m
  return cost



def Generalization(weights, lambda_):
  cost = 0
  for i in range(len(weights)):
    cost += weights[i] ** 2

  cost *= lambda_ / (2 * len(weights))

  return cost



def GradientDescent(X, Y, weights, b, lambda_):
    delta_w = np.zeros(len(weights))
    delta_b = 0

    m = len(Y)

    for i in range(m):
        x = X[i]
        y = Y[i]

        h = Sigmoid(np.dot(x, weights) + b)
        delta_w += (h - y) * x
        delta_b += (h - y)

    delta_w /= m
    delta_b /= m

    delta_w += lambda_ * weights / m

    return delta_w, delta_b

def accuracy(X, Y, weights, b):
  m = len(Y)
  correct = 0

  for i in range(m):
      x = X[i]
      y = Y[i]

      h = Sigmoid(np.dot(x, weights) + b)

      if h >= 0.5:
          prediction = 1
      else:
          prediction = 0

      if prediction == y:
          correct += 1

  accuracy = correct / m
  return accuracy



def Training(X_train_scaled, Y_train, weights, b, learning_rate, lambda_, epochs):
    cost_history = []

    # 1 epoch = 1000 iterations
    for i in range(epochs):
        # Initialize cost for this epoch
        curr_cost = CostFunction(X_train_scaled, Y_train, weights, b) + Generalization(weights, lambda_)

        # Store the cost once per epoch
        cost_history.append(curr_cost)

        # Perform 1000 iterations per epoch
        for j in range(1000):
            delta_w, delta_b = GradientDescent(X_train_scaled, Y_train, weights, b, lambda_)

            # Update weights and bias
            weights -= learning_rate * delta_w
            b -= learning_rate * delta_b

        # Output cost at the end of each epoch for tracking
        print(f'Epoch: {i + 1}, Cost: {curr_cost}')

        # Print out model accuracy:
        print(f'Accuracy on Trained Data: {accuracy(X_train_scaled, Y_train, weights, b)}')
        print(f'Accuracy on Test Data: {accuracy(X_test_scaled, Y_test, weights, b)}')

    return weights, b, cost_history


In [69]:
weights = np.zeros(columns)
b = 0.0
weights, b, cost_history = Training(X_train_scaled, Y_train, weights, b, 0.01, 0.1, 100)

Epoch: 1, Cost: 0.6931471805599468
Accuracy on Trained Data: 0.9296703296703297
Accuracy on Test Data: 0.9298245614035088
Epoch: 2, Cost: 0.18138359733646375
Accuracy on Trained Data: 0.9296703296703297
Accuracy on Test Data: 0.9298245614035088
Epoch: 3, Cost: 0.17281346749960586
Accuracy on Trained Data: 0.9296703296703297
Accuracy on Test Data: 0.9298245614035088
Epoch: 4, Cost: 0.17097777851204665
Accuracy on Trained Data: 0.9296703296703297
Accuracy on Test Data: 0.9298245614035088
Epoch: 5, Cost: 0.17043365568186875
Accuracy on Trained Data: 0.9318681318681319
Accuracy on Test Data: 0.9298245614035088
Epoch: 6, Cost: 0.17023467073573922
Accuracy on Trained Data: 0.9318681318681319
Accuracy on Test Data: 0.9298245614035088
Epoch: 7, Cost: 0.17013878006825134
Accuracy on Trained Data: 0.9340659340659341
Accuracy on Test Data: 0.9385964912280702
Epoch: 8, Cost: 0.17007580831043523
Accuracy on Trained Data: 0.9362637362637363
Accuracy on Test Data: 0.9385964912280702
Epoch: 9, Cost: 0

In [70]:
"""FINAL WEIGHTS AND BIAS OF MODEL"""
print(weights)
print(b)

[ 0.          0.5445582   0.87350228  0.25748292  0.68734925  1.13137091
  0.45471438 -0.41140685  0.97829189  0.0910372   0.77823419  1.14284017
  0.56235006  0.3930373   1.02080871  0.30212809 -1.62707896 -0.204172
  0.6666311  -0.10613133]
-0.53689376245296
